In [ ]:
import requests
from bs4 import BeautifulSoup
url = 'https://developers.naver.com/docs/serviceapi/search/book/book.md#%EC%B1%85'

#dict 타입으로 요청 파라미터 설정
req_param = {
    'sid': 105
}
# 
url = 'https://developers.naver.com/docs/serviceapi/search/book/book.md/{sid}'.format(**req_param)
print(url)

# 요청 헤더 설정 : 브라우저 정보( 사람처럼 보이게 하기 위함)
req_header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36'
}

# requests 의 get() 함수 호출하기 
res = requests.get(url, headers=req_header)
print(res.status_code)
print(res.ok)
print(type(res))
#print(res.text)
# 응답(response)이 OK 이면
# 응답 (response)에서 text 추출
# BeautifulSoup 객체 생성  
if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    # print(soup.select("div.sa_text a[href*='mnews/article']"))
    print(len(soup.select("div.sa_text a[href='https://n.news.naver.com/mnews/']")))
    #CSS 선택자를 사용해서 a tag 목록 가져오기
    a_tags = soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/']")
    # <a> 태그 리스트 순회하기 
    print(type(a_tags), type(a_tags[0])) # [Tag, Tag]
    for a_tag in a_tags:
        title = a_tag.text.strip()
        link = a_tag['href']
        print(title, link)
else:
    # 응답(response)이 Error 이면 status code 출력   
    print(f'Error: {res.status_code}')


In [ ]:
# 1. 질문 : 검색어로 찾은 책 목록을 json 파일로 저장하기
def search_books(query):   
    # query string 문자열을 dict 선언
    payload = {
    'query':  query, #'파이썬',
    'display': 100,
    'sort': 'sim'
    }


    url = 'https://openapi.naver.com/v1/search/book.json'
    
    # requests get 요청
    res = requests.get(url, params=payload, headers=headers)
    
    if res.status_code == 200:
        # json() 함수로 응답 결과 가져오기
        items_data = res.json()['items']
        
        # data 폴더가 없다면 생성 (선택사항)
        import os
        if not os.path.exists('data'):
            os.makedirs('data')
            
        # data/books.json 파일로 저장
        with open('data/books.json', 'w', encoding='utf-8') as f:
            json.dump(items_data, f, indent=4, ensure_ascii=False)
            
        return items_data
    else:
        print(f"Error Code: {res.status_code}")
        return None

# '파이썬' 검색어로 실행
items = search_books('파이썬')

Error Code: 401


In [ ]:
# 파일에서 읽어와 데이터프레임 생성
with open('data/books.json', 'r', encoding='utf-8') as f:
    books_dict = json.load(f)

df = pd.DataFrame(books_dict)

# 결과 확인
print(f"전체 데이터 개수: {len(df)}")

In [ ]:
# 전체 목록 출력 (상위 5개 예시)
df.head()

In [ ]:
# discount 컬럼을 숫자형으로 변환 (비어있는 경우 0 처리)
df['discount'] = pd.to_numeric(df['discount'], errors='coerce').fillna(0).astype(int)

# 조건: 가격이 20,000원 이상
over_20k_df = df[df['discount'] >= 20000].copy()

# title, author, discount, publisher, pubdate 컬럼만 선택
over_20k_df = over_20k_df[['title', 'author', 'discount', 'publisher', 'pubdate']]

# 가격 기준 내림차순 정렬 및 인덱스 초기화
over_20k_df = over_20k_df.sort_values(by='discount', ascending=False).reset_index(drop=True)

# 출력
over_20k_df

In [ ]:
import requests

upload_files = {
    'img1': open('img/f1.jpg','rb'),
    'img2': open('img/f2.jpg','rb'),
}
#print(upload_files)

url = 'http://httpbin.org/post'
# file 업로드 하려면 requests의 post 함수에 files 속성을 사용합니다.
res = requests.post(url, files=upload_files)
print(res.status_code)
print(res.json()['files']['img1'])

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

CLIENT_ID = os.getenv("CLIENT_ID")
print(CLIENT_ID[:4])

CLIENT_SECRET = os.getenv("CLIENT_SECRET")
print(CLIENT_SECRET[:4])

In [ ]:
# 조건: 출판사가 "인피니티북스"
infinity_df = df[df['publisher'] == '인피니티북스'].copy()

# image, description 컬럼 제외 (drop 함수 사용)
infinity_df = infinity_df.drop(columns=['image', 'description'])

# 인덱스 초기화
infinity_df = infinity_df.reset_index(drop=True)

# 출력
infinity_df